# **Ad copy**

Zero shot prompting.



### 1. Setup requirements and dependencies




In [2]:
%pip install transformers==4.38.2 --quiet
%pip install datasets==2.18.0  --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.5/8.5 MB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 43.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 6.2 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset

### 2. Load the sample hugging face dataset

In [4]:
huggingface_ad_copy_dataset = "jaykin01/advertisement-copy"
dataset = load_dataset(huggingface_ad_copy_dataset)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

In [16]:
for i in range(0, 3):
  print(dataset['train'][i]["product"])
  print(dataset['train'][i]["description"])
  print(dataset['train'][i]["ad"])

  print("********************************")

 Harem pants
 A style of pants with a dropped crotch, loose-fitting legs, and a gathered waistband for a unique, bohemian look.
Discover Harem Pants! Unique, stylish bohemian vibes with a dropped crotch & loose legs. Comfy meets chic - elevate your wardrobe. Limited stock - shop now!
********************************
 Fringe skirt
 A skirt featuring fringe detailing on the bottom, creating movement and fun.
Introducing our fabulous Fringe Skirt! Step out in style with eye-catching fringe detailing that adds flair and movement. Perfect for any occasion, create unforgettable memories with this chic piece.
********************************
 Gingham dress
 A dress featuring a two-toned checkered pattern, often associated with picnics and summery outfits.
Introducing the Gingham Dress: Timeless & Chic! 💕 Step into summer with this must-have, two-toned checkered dress. From picnics to parties, it's your go-to look. Shop now for unbeatable style!
********************************


In [ ]:
from transformers import pipeline, set_seed
generator = pipeline('text-generation', model='google/flan-t5-base')
set_seed(42)
generator("Hello, I'm a language model,", max_length=30, num_return_sequences=5)


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "Hello, I'm a language model, but what I'm really doing is making a human-readable document. There are other languages, but those are"},
 {'generated_text': "Hello, I'm a language model, not a syntax model. That's why I like it. I've done a lot of programming projects.\n"},
 {'generated_text': "Hello, I'm a language model, and I'll do it in no time!\n\nOne of the things we learned from talking to my friend"},
 {'generated_text': "Hello, I'm a language model, not a command line tool.\n\nIf my code is simple enough:\n\nif (use (string"},
 {'generated_text': "Hello, I'm a language model, I've been using Language in all my work. Just a small example, let's see a simplified example."}]

In [8]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
import torch

tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

ENCODED SENTENCE:
tensor([8774,    6,   82, 1782,   19, 5295,    1])

DECODED SENTENCE:
Hello, my dog is cute


In [19]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
tokenizer.pad_token = '[PAD]'


example_indices = [100, 110]
seperator = '*'.join('' for x in range(100))
for i, index in enumerate(example_indices):

  if index < len(dataset['train']):
    description = dataset['train'][index]["description"]
    ad = dataset['train'][index]["ad"]

    inputs = tokenizer(description, return_tensors='pt', padding=True)
    input_ids = inputs["input_ids"]
    output = tokenizer.decode(
                model.generate(
                    input_ids,
                    max_new_tokens=50,
                )[0],
                skip_special_tokens=True
      )
    print("-----------------------------------------------")
    print('Example ', i + 1)
    print(seperator)
    print(f'INPUT PROMPT:\n{description}')
    print(seperator)
    print(f'BASELINE AD:\n{ad}')
    print(seperator)
    print(f'MODEL GENERATION - WITHOUT PROMPT ENGINEERING:\n{output}\n')

-----------------------------------------------
Example  1
***************************************************************************************************
INPUT PROMPT:
 A wide, shallow neckline that extends from shoulder to shoulder, creating a classic and feminine look.
***************************************************************************************************
BASELINE AD:
Shop now! Discover elegance redefined with our timeless Boatneck Blouse - the perfect blend of a classic wide neckline & feminine charm, crafted for your ultimate style statement!
***************************************************************************************************
MODEL GENERATION - WITHOUT PROMPT ENGINEERING:
A wide, shallow neckline that extends from shoulder to shoulder, creating a classic and feminine look.

-----------------------------------------------
Example  2
***************************************************************************************************
INPUT PROMPT:
 A cas